In [1]:
from google.colab import output
output.enable_custom_widget_manager()

In [2]:
# ==== INSTALARE DEPENDENȚE ====
# Rulează această celulă prima dată
!pip install ifcopenshell requests plotly ipywidgets trimesh


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.5/736.5 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import trimesh

# Calea către fișierul GLB din Google Drive
glb_path = '/content/drive/My Drive/qto_tests/glb/test_geometry.glb'

# Încarcă fișierul GLB
mesh = trimesh.load(glb_path)

# Afișează informații despre model (opțional)
print(mesh)

<trimesh.Scene(len(geometry)=66)>


In [5]:
import json

# Calea către fișierul JSON din Google Drive
json_path = '/content/drive/My Drive/qto_tests/glb/test_metadata.json'

# Citește fișierul JSON
with open(json_path, 'r') as file:
    json_data = json.load(file)

# Afișează conținutul JSON (opțional)
print(json_data)

{'elements': {'1': {'id': 1, 'parent_id': None, 'GlobalId': '21MDAeADb9799Web$9RSGv', 'IfcEntity': 'IfcProject', 'Classifications': [], 'Systems': [], 'Name': 'Mustermodell V1'}, '2': {'id': 2, 'parent_id': 164, 'GlobalId': '0tkgbUMcT25OBOhCgbQt78', 'IfcEntity': 'IfcCovering', 'Classifications': [], 'Systems': [], 'ConnectedFrom': [], 'ConnectedTo': [], 'ContainedInStructure': ['3_tSJLGEDBgPkOwRkkHz8p'], 'CoversElements': ['2K$Hz7MqP9He8Rys5yW94b'], 'CoversSpaces': [], 'FillsVoids': [], 'HasOpenings': [], 'HasProjections': [], 'InterferesElements': [], 'IsConnectionRealization': [], 'IsInterferedByElements': [], 'Name': 'FLOORING', 'PredefinedType': 'FLOORING', 'ProvidesBoundaries': [], 'ReferencedInStructures': [], 'Pset_CoveringCommon.IsExternal': False, 'ePset_abstractBIM.PredefinedType': 'FLOORING', 'ePset_abstractBIM.IsExternal': False, 'ePset_abstractBIM.Spaces': '1QkKxMqDv7nuSnClC0HTpb', 'ePset_abstractBIM.SpacesLongName': 'SGR', 'ePset_abstractBIM.SpacesName': 'SGR', 'ePset_abs

In [6]:
import ifcopenshell
import requests
import json
import numpy as np
import plotly.graph_objects as go
from IPython.display import display
import ipywidgets as widgets
from scipy.spatial.transform import Rotation
from collections import defaultdict
import traceback

 # ==== IFC DOWNLOAD AND PROCESSING MODULE ====
class IFCDownloader:
    """Handles downloading and loading IFC files."""
    @staticmethod
    def download_and_load(url):
        try:
            #print("🔄 Downloading IFC file...")
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            with open("temp.ifc", "wb") as f:
                f.write(response.content)
            model = ifcopenshell.open("temp.ifc")
                #print(f"✅ IFC file downloaded successfully! Size: {len(response.content)} bytes")
                #print(f"📊 IFC Schema: {model.schema}")
                #print(f"📊 Total entities: {len(list(model))}")
            return model
        except requests.RequestException as e:
                #print(f"❌ Error downloading file: {e}")
            raise
        except Exception as e:
                #print(f"❌ Error processing IFC file: {e}")
            raise

 # ==== GEOMETRY EXTRACTION MODULE ====
class GeometryExtractor:
    """Extracts geometry from Custom_Mesh properties and QTO properties."""
    @staticmethod
    def extract_custom_mesh_from_entity(entity):
        if not hasattr(entity, 'IsDefinedBy'):
            return None
        for rel in entity.IsDefinedBy:
            if rel.is_a("IfcRelDefinesByProperties"):
                pset = rel.RelatingPropertyDefinition
                if hasattr(pset, 'Name') and pset.Name == "Pset_CustomGeometry":
                    if hasattr(pset, 'HasProperties'):
                        for prop in pset.HasProperties:
                            if hasattr(prop, 'Name') and prop.Name == "Custom_Mesh":
                                if hasattr(prop, 'NominalValue') and prop.NominalValue:
                                    return prop.NominalValue.wrappedValue
        return None

    @staticmethod
    def extract_qto_properties(entity, model):
        """Extracts all QTO properties from IfcElementQuantity sets only for elements with geometry."""
        qto_props = {}
    # Check if the element has geometry
        if not GeometryExtractor.extract_custom_mesh_from_entity(entity):
            #print(f"⚠️ {entity.GlobalId} ({entity.is_a()}): No Custom_Mesh, skipping QTO")
            return qto_props

    # Extract QTO properties of the main element
        if hasattr(entity, 'IsDefinedBy'):
            for rel in entity.IsDefinedBy:
                if rel.is_a("IfcRelDefinesByProperties"):
                    pset = rel.RelatingPropertyDefinition
                    if pset.is_a("IfcElementQuantity") and hasattr(pset, 'Name') and pset.Name.startswith("Qto_"):
                        #print(f"📏 {entity.GlobalId} ({entity.is_a()}): Found set {pset.Name}")
                        if hasattr(pset, 'Quantities'):
                            if not pset.Quantities:
                                pass
                                #print(f"⚠️ {entity.GlobalId} ({entity.is_a()}): Set {pset.Name} is empty")
                            for qty in pset.Quantities:
                                qty_name = getattr(qty, 'Name', 'Unknown')
                                qty_value = None
                                # Accesăm atributul specific în funcție de tipul de cantitate
                                if qty.is_a('IfcQuantityLength'):
                                    qty_value = getattr(qty, 'LengthValue', None)
                                elif qty.is_a('IfcQuantityArea'):
                                    qty_value = getattr(qty, 'AreaValue', None)
                                elif qty.is_a('IfcQuantityVolume'):
                                    qty_value = getattr(qty, 'VolumeValue', None)
                                elif qty.is_a('IfcQuantityCount'):
                                    qty_value = getattr(qty, 'CountValue', None)
                                elif qty.is_a('IfcQuantityWeight'):
                                    qty_value = getattr(qty, 'WeightValue', None)

                                #print(f"    - {qty_name}: Type={qty.is_a()}, Value={qty_value}")
                                if qty_value is not None:
                                    qto_props[qty_name] = qty_value
                                    #print(f"    - {qty_name}: {qty_value} (direct value)")
                                else:
                                    pass
                                    #print(f"    - {qty_name}: No valid value")

    # Check associated elements (e.g. IfcCovering for IfcWall)
        if entity.is_a("IfcWall") or entity.is_a("IfcWallStandardCase"):
            for rel in model.by_type("IfcRelCoversBldgElements"):
                if rel.RelatingBuildingElement == entity and rel.RelatedCoverings:
                    for covering in rel.RelatedCoverings:
                        if GeometryExtractor.extract_custom_mesh_from_entity(covering):
                            #print(f"📏 {entity.GlobalId} ({entity.is_a()}): Found associated IfcCovering {covering.GlobalId}")
                            if hasattr(covering, 'IsDefinedBy'):
                                for rel_cov in covering.IsDefinedBy:
                                    if rel_cov.is_a("IfcRelDefinesByProperties"):
                                        pset = rel_cov.RelatingPropertyDefinition
                                        if pset.is_a("IfcElementQuantity") and hasattr(pset, 'Name') and pset.Name.startswith("Qto_"):
                                            #print(f"📏 {entity.GlobalId} ({entity.is_a()}): Found set {pset.Name} for IfcCovering {covering.GlobalId}")
                                            if hasattr(pset, 'Quantities'):
                                                if not pset.Quantities:
                                                    pass
                                                    #print(f"⚠️ {entity.GlobalId} ({entity.is_a()}): Set {pset.Name} is empty for IfcCovering")
                                                for qty in pset.Quantities:
                                                    qty_name = getattr(qty, 'Name', 'Unknown')
                                                    qty_value = None
                                                    if qty.is_a('IfcQuantityLength'):
                                                        qty_value = getattr(qty, 'LengthValue', None)
                                                    elif qty.is_a('IfcQuantityArea'):
                                                        qty_value = getattr(qty, 'AreaValue', None)
                                                    elif qty.is_a('IfcQuantityVolume'):
                                                        qty_value = getattr(qty, 'VolumeValue', None)
                                                    elif qty.is_a('IfcQuantityCount'):
                                                        qty_value = getattr(qty, 'CountValue', None)
                                                    elif qty.is_a('IfcQuantityWeight'):
                                                        qty_value = getattr(qty, 'WeightValue', None)
                                                    #print(f"    - {qty_name}_Covering: Type={qty.is_a()}, Value={qty_value}")
                                                    if qty_value is not None:
                                                        qto_props[f"{qty_name}_Covering"] = qty_value
                                                        #print(f"    - {qty_name}_Covering: {qty_value} (direct value)")
                                                    else:
                                                        pass
                                                        #print(f"    - {qty_name}_Covering: No valid value")

        if qto_props:
            pass
            #print(f"📏 {entity.GlobalId} ({entity.is_a()}): Extracted QTO properties: {qto_props}")
        else:
            pass
            #print(f"⚠️ {entity.GlobalId} ({entity.is_a()}): No valid QTO property found")
        return qto_props

    @staticmethod
    def transform_coordinates(vertices, rotation, translation):
        vertices = np.array(vertices, dtype=np.float32).reshape(-1, 3)
        if rotation and any(rotation.values()):
            quat = [rotation["qw"], rotation["qx"], rotation["qy"], rotation["qz"]]
            rot = Rotation.from_quat(quat)
            vertices = rot.apply(vertices)
        vertices = vertices[:, [0, 2, 1]]
        position = [translation["x"], translation["z"], translation["y"]]
        vertices += position
        return vertices

 # ==== HIERARCHICAL STRUCTURE MODULE ====
class HierarchicalStructure:
    def __init__(self, model):
        self.model = model
        self.hierarchy = defaultdict(lambda: defaultdict(list))
        self.build_hierarchy()

    def build_hierarchy(self):
    #print("🏗️ Building hierarchical structure...")
        building_storeys = self.model.by_type("IfcBuildingStorey")
        if not building_storeys:
            #print("⚠️ No Building Storeys found. Using default structure.")
            self._build_default_hierarchy()
            return
        for storey in building_storeys:
            storey_name = storey.Name or f"Storey_{storey.GlobalId[:8]}"
            if hasattr(storey, 'ContainsElements'):
                for rel in storey.ContainsElements:
                    if rel.is_a("IfcRelContainedInSpatialStructure"):
                        for element in rel.RelatedElements:
                            if element.is_a("IfcProduct"):
                                ifc_type = element.is_a()
                                self.hierarchy[storey_name][ifc_type].append(element)
        if not any(self.hierarchy.values()):
            #print("⚠️ No elements found in Building Storeys. Trying direct mapping...")
            self._build_fallback_hierarchy()
            return
        total_storeys = len(self.hierarchy)
        total_types = sum(len(types) for types in self.hierarchy.values())
        total_elements = sum(len(elements) for types in self.hierarchy.values() for elements in types.values())
    #print(f"✅ Structure built: {total_storeys} storeys, {total_types} types, {total_elements} elements")

    def _build_fallback_hierarchy(self):
    #print("🔄 Building fallback structure...")
        elements_with_mesh = []
        all_products = self.model.by_type("IfcProduct")
        for element in all_products:
            if GeometryExtractor.extract_custom_mesh_from_entity(element):
                elements_with_mesh.append(element)
    #print(f"🔍 Found {len(elements_with_mesh)} elements with Custom_Mesh")
        building_storeys = self.model.by_type("IfcBuildingStorey")
        storey_mapping = {}
        for element in elements_with_mesh:
            assigned_storey = None
            for storey in building_storeys:
                if hasattr(storey, 'ContainsElements'):
                    for rel in storey.ContainsElements:
                        if rel.is_a("IfcRelContainedInSpatialStructure"):
                            if element in rel.RelatedElements:
                                assigned_storey = storey
                                break
                if assigned_storey:
                    break
            if not assigned_storey:
                if building_storeys:
                    assigned_storey = building_storeys[0]
                else:
                    assigned_storey = None
            if assigned_storey:
                storey_name = assigned_storey.Name or f"Storey_{assigned_storey.GlobalId[:8]}"
            else:
                storey_name = "Default_Level"
            ifc_type = element.is_a()
            self.hierarchy[storey_name][ifc_type].append(element)

    def _build_default_hierarchy(self):
    #print("🔄 Building default structure...")
        all_products = self.model.by_type("IfcProduct")
        for element in all_products:
            if GeometryExtractor.extract_custom_mesh_from_entity(element):
                storey_name = "Default_Level"
                ifc_type = element.is_a()
                self.hierarchy[storey_name][ifc_type].append(element)
    #print(f"🔍 Found {sum(len(elements) for types in self.hierarchy.values() for elements in types.values())} elements with Custom_Mesh")

    def get_hierarchy(self):
        return dict(self.hierarchy)

    #def print_hierarchy_summary(self):
        #print("\n📋 HIERARCHICAL STRUCTURE:")
        #print("=" * 60)
        #for storey_name, types in self.hierarchy.items():
        #    print(f"\n🏢 {storey_name}")
        #    print("-" * 40)
        #    for ifc_type, elements in types.items():
        #        print(f"  📦 {ifc_type}: {len(elements)} elements")
        #        for i, element in enumerate(elements[:3]):
        #            name = element.Name or f"Element_{element.GlobalId[:8]}"
        #            print(f"    • {name}")
        #        if len(elements) > 3:
        #            print(f"    ... and {len(elements) - 3} more elements")

 # ==== 3D VISUALIZATION MODULE ====
class Visualizer3D:
    def __init__(self):
        self.fig = go.Figure()
        self.mesh_dict = {}
        self.original_colors = {}
        self.visibility = {}
        self.properties = {}
        self.selected_mesh = [None]

    def add_mesh_from_element(self, element, mesh_json, hierarchy_path, qto_props):
        try:
            mesh_data = json.loads(mesh_json)
            for element_data in mesh_data["elements"]:
                mesh_id = element_data["mesh_id"]
                mesh_info = next((m for m in mesh_data["meshes"] if m["mesh_id"] == mesh_id), None)
                if not mesh_info:
                    continue
                vertices = GeometryExtractor.transform_coordinates(
                    mesh_info["coordinates"],
                    element_data["rotation"],
                    element_data["vector"]
                )
                indices = np.array(mesh_info["indices"], dtype=np.uint32)
                element_name = element.Name or f"{element.is_a()}_{element.GlobalId[:8]}"
                full_name = f"{hierarchy_path}/{element_name}"
                r, g, b = element_data["color"]["r"], element_data["color"]["g"], element_data["color"]["b"]
                hex_color = f"#{r:02x}{g:02x}{b:02x}"
                mesh = go.Mesh3d(
                    x=vertices[:, 0],
                    y=vertices[:, 1],
                    z=vertices[:, 2],
                    i=indices[0::3],
                    j=indices[1::3],
                    k=indices[2::3],
                    color=hex_color,
                    opacity=element_data["color"]["a"] / 255,
                    name=full_name,
                    visible=True
                )
                self.fig.add_trace(mesh)
                self.mesh_dict[full_name] = mesh
                self.original_colors[full_name] = hex_color
                self.visibility[full_name] = True
                self.properties[full_name] = {
                    "Name": element_name,
                    "Type": element.is_a(),
                    "GUID": element.GlobalId,
                    "Hierarchy": hierarchy_path,
                    **element_data["info"],
                    **qto_props
                }
                #print(f"✅ Mesh added: {full_name} (Total meshes: {len(self.mesh_dict)})")
        except Exception as e:
            print(f"⚠️ Error processing mesh for {element.GlobalId}: {e}")

    def configure_layout(self):
        self.fig.update_layout(
            scene=dict(
                xaxis=dict(title="X"),
                yaxis=dict(title="Y (Vertical)"),
                zaxis=dict(title="Z"),
                aspectmode="data",
                camera=dict(
                    up=dict(x=0, y=1, z=0),
                    eye=dict(x=1.5, y=1.5, z=1.5)
                )
            ),
            showlegend=False,
            width=800,
            height=600,
            margin=dict(l=0, r=0, t=0, b=0)
        )

 # ==== USER INTERFACE MODULE WITH TABLE AND VISIBILITY PANEL ====
class HierarchicalTableUI:
    def __init__(self, hierarchy, visualizer, model):
        self.hierarchy = hierarchy
        self.visualizer = visualizer
        self.model = model
        self.properties_panel = widgets.HTML(value="<b>Properties:</b><br>Select an object")
        self.all_checkboxes = {}
        self.table_output = widgets.Output()
        self.viewer_output = widgets.Output()
        self.filter_storey = None
        self.filter_ifc_type = None

    def create_ui(self):
        print("🔧 Starting UI creation with filters...")
        storey_options = ['All'] + sorted(list(self.hierarchy.keys()))
        ifc_type_options = ['All'] + sorted(set(ifc_type for storey in self.hierarchy.values() for ifc_type in storey.keys()))
        self.storey_dropdown = widgets.Dropdown(
                options=storey_options,
                value='All',
                description='Storey:',
                layout=widgets.Layout(width='300px')
            )
        self.ifc_type_dropdown = widgets.Dropdown(
                options=ifc_type_options,
                value='All',
                description='IFC Type:',
                layout=widgets.Layout(width='300px')
            )
        apply_filter_btn = widgets.Button(
                description='Apply Filters',
                button_style='success',
                layout=widgets.Layout(width='120px')
            )
        apply_filter_btn.on_click(self._apply_filters)
        filter_box = widgets.HBox([self.storey_dropdown, self.ifc_type_dropdown, apply_filter_btn])
        visibility_accordion = self._create_visibility_accordion()
        expand_all_btn = widgets.Button(
            description="Expand all",
            button_style='info',
            layout=widgets.Layout(width='120px')
        )
        collapse_all_btn = widgets.Button(
            description="Collapse all",
            button_style='info',
            layout=widgets.Layout(width='120px')
        )
        expand_all_btn.on_click(lambda x: self._expand_all_accordions(visibility_accordion))
        collapse_all_btn.on_click(lambda x: self._collapse_all_accordions(visibility_accordion))
        buttons_box = widgets.HBox([expand_all_btn, collapse_all_btn])
        visibility_panel = widgets.VBox([
            widgets.HTML("<b>🎛️ Element Visibility Control</b>"),
            buttons_box,
            visibility_accordion
        ], layout=widgets.Layout(
            border='1px solid #ddd',
            padding='10px',
            max_height='400px',
            overflow_y='auto'
        ))
        control_panel = widgets.VBox([
            widgets.HTML("<b>📊 IFC Elements Filters and Table</b>"),
            filter_box,
            self.viewer_output,
            self.table_output,
            self.properties_panel,
            visibility_panel
        ])
        self._update_table()
        self._update_viewer()
        print(f"✅ UI created with filters, table, and dynamic viewer")
        return control_panel

    def _create_table(self):
        header_values = ["Storey", "Type", "Name", "GUID"]
        qto_keys_by_type = defaultdict(set)
        for storey_name, types in self.hierarchy.items():
            if self.filter_storey and self.filter_storey != 'All' and storey_name != self.filter_storey:
                continue
            for ifc_type, elements in types.items():
                if self.filter_ifc_type and self.filter_ifc_type != 'All' and ifc_type != self.filter_ifc_type:
                    continue
                for element in elements:
                    if GeometryExtractor.extract_custom_mesh_from_entity(element):
                        qto_props = GeometryExtractor.extract_qto_properties(element, self.model)
                        qto_keys_by_type[ifc_type].update(qto_props.keys())
        qto_keys = sorted(set().union(*qto_keys_by_type.values()))
        if qto_keys:
            print(f"📏 QTO keys found: {qto_keys}")
            header_values.extend(qto_keys)
        else:
            print("⚠️ No QTO key found")
        cells_values = [[] for _ in header_values]
        row_colors = []
        current_row = 0
        for storey_name, types in self.hierarchy.items():
            if self.filter_storey and self.filter_storey != 'All' and storey_name != self.filter_storey:
                continue
            for ifc_type, elements in types.items():
                if self.filter_ifc_type and self.filter_ifc_type != 'All' and ifc_type != self.filter_ifc_type:
                    continue
                for element in elements:
                    if GeometryExtractor.extract_custom_mesh_from_entity(element):
                        element_name = element.Name or f"{element.is_a()}_{element.GlobalId[:8]}"
                        full_name = f"{storey_name}/{ifc_type}/{element_name}"
                        if full_name in self.visualizer.mesh_dict:
                            cells_values[0].append(storey_name)
                            cells_values[1].append(ifc_type)
                            cells_values[2].append(element_name)
                            cells_values[3].append(element.GlobalId[:8])
                            qto_props = GeometryExtractor.extract_qto_properties(element, self.model)
                            for key in qto_keys:
                                value = qto_props.get(key, "-")
                                cells_values[header_values.index(key)].append(str(value))
                            row_colors.append("#ffffff" if current_row % 2 == 0 else "#f0f0f0")
                            current_row += 1
        if not cells_values[0]:
            return go.Figure(data=[go.Table(
                header=dict(values=["Message"], fill_color='#FF5733', font=dict(color='white')),
                cells=dict(values=[["No element matches the filters."]], align='center')
            )])
        table = go.Figure(data=[go.Table(
            header=dict(
                values=header_values,
                fill_color='#4CAF50',
                font=dict(color='white', size=12),
                align='left'
            ),
            cells=dict(
                values=cells_values,
                fill_color=[row_colors],
                align='left',
                height=30
            )
        )])
        table.update_layout(
            width=800,
            height=400,
            margin=dict(l=0, r=0, t=20, b=0),
            clickmode='event+select'
        )
        def on_table_click(trace, points, selector):
            if points.point_inds:
                row_index = points.point_inds[0]
                full_name = f"{cells_values[0][row_index]}/{cells_values[1][row_index]}/{cells_values[2][row_index]}"
                self._select_mesh(full_name)
        table.data[0].on_click(on_table_click)
        return table

    def _update_table(self):
        with self.table_output:
            self.table_output.clear_output()
            table = self._create_table()
            display(table)

    def _update_viewer(self):
        with self.viewer_output:
            self.viewer_output.clear_output()
            for i, trace in enumerate(self.visualizer.fig.data):
                if trace.name in self.visualizer.visibility:
                    self.visualizer.fig.data[i].visible = self.visualizer.visibility[trace.name]
            display(self.visualizer.fig)

    def _apply_filters(self, b):
        self.filter_storey = self.storey_dropdown.value
        self.filter_ifc_type = self.ifc_type_dropdown.value
        print(f"🔍 Applying filters: Storey={self.filter_storey}, IFC Type={self.filter_ifc_type}")
        for full_name in self.visualizer.mesh_dict:
            storey, ifc_type, _ = full_name.split('/')
            is_visible = (
                (self.filter_storey == 'All' or storey == self.filter_storey) and
                (self.filter_ifc_type == 'All' or ifc_type == self.filter_ifc_type)
            )
            self.visualizer.visibility[full_name] = is_visible
            self.visualizer.mesh_dict[full_name].visible = is_visible
            if full_name in self.all_checkboxes:
                self.all_checkboxes[full_name].value = is_visible
        self._update_table()
        self._update_viewer()
        print("✅ 3D view and table updated")

    def _create_visibility_accordion(self):
        visibility_accordion = widgets.Accordion(children=[])
        visibility_titles = []
        for storey_name, types in self.hierarchy.items():
            if self.filter_storey and self.filter_storey != 'All' and storey_name != self.filter_storey:
                continue
            type_accordion = widgets.Accordion(children=[])
            type_titles = []
            for ifc_type, elements in types.items():
                if self.filter_ifc_type and self.filter_ifc_type != 'All' and ifc_type != self.filter_ifc_type:
                    continue
                element_checkboxes = []
                for element in elements:
                    if GeometryExtractor.extract_custom_mesh_from_entity(element):
                        element_name = element.Name or f"{element.is_a()}_{element.GlobalId[:8]}"
                        full_name = f"{storey_name}/{ifc_type}/{element_name}"
                        if full_name in self.visualizer.mesh_dict:
                            checkbox = widgets.Checkbox(
                                value=self.visualizer.visibility.get(full_name, True),
                                description=element_name,
                                indent=False,
                                layout=widgets.Layout(width='auto')
                            )
                            checkbox.full_name = full_name
                            checkbox.observe(self._on_checkbox_change, names="value")
                            element_checkboxes.append(checkbox)
                            self.all_checkboxes[full_name] = checkbox
                if element_checkboxes:
                    select_all_checkbox = widgets.Checkbox(
                        value=True,
                        description=f"Select all ({len(element_checkboxes)} elements)",
                        indent=False,
                        style={'font_weight': 'bold'}
                    )
                    select_all_checkbox.related_checkboxes = element_checkboxes
                    select_all_checkbox.observe(self._on_select_all_change, names="value")
                    type_container = widgets.VBox([
                        select_all_checkbox,
                        widgets.HTML("<hr style='margin: 5px 0;'>"),
                        *element_checkboxes
                    ])
                    type_accordion.children += (type_container,)
                    type_titles.append(f"{ifc_type} ({len(element_checkboxes)})")
            if type_accordion.children:
                for i, title in enumerate(type_titles):
                    type_accordion.set_title(i, title)
                visibility_accordion.children += (type_accordion,)
                visibility_titles.append(f"{storey_name} ({len(type_titles)} types)")
        for i, title in enumerate(visibility_titles):
            visibility_accordion.set_title(i, title)
        return visibility_accordion

    def _on_checkbox_change(self, change):
        checkbox = change['owner']
        full_name = checkbox.full_name
        is_visible = change['new']
        self.visualizer.visibility[full_name] = is_visible
        self.visualizer.mesh_dict[full_name].visible = is_visible
        for i, trace in enumerate(self.visualizer.fig.data):
            if trace.name == full_name:
                self.visualizer.fig.data[i].visible = is_visible
                break
        self._update_viewer()
        if is_visible:
            self._select_mesh(full_name)
        else:
            if self.visualizer.selected_mesh[0] == full_name:
                self._deselect_current()

    def _on_select_all_change(self, change):
        select_all_checkbox = change['owner']
        new_value = change['new']
        for checkbox in select_all_checkbox.related_checkboxes:
            if checkbox.full_name in self.visualizer.mesh_dict:
                checkbox.value = new_value
                self.visualizer.visibility[checkbox.full_name] = new_value
                self.visualizer.mesh_dict[checkbox.full_name].visible = new_value
                for i, trace in enumerate(self.visualizer.fig.data):
                    if trace.name == checkbox.full_name:
                        self.visualizer.fig.data[i].visible = new_value
                        break
        self._update_viewer()

    def _select_mesh(self, full_name):
        if full_name not in self.visualizer.mesh_dict:
            return
        self.visualizer.selected_mesh[0] = full_name
        for i, trace in enumerate(self.visualizer.fig.data):
            color = "#ffff00" if trace.name == full_name else self.visualizer.original_colors[trace.name]
            self.visualizer.fig.data[i].color = color
        self._update_viewer()
        element = None
        for storey_name, types in self.hierarchy.items():
            for ifc_type, elements in types.items():
                for elem in elements:
                    element_name = elem.Name or f"{elem.is_a()}_{elem.GlobalId[:8]}"
                    if full_name == f"{storey_name}/{ifc_type}/{element_name}":
                        element = elem
                        break
                if element:
                    break
            if element:
                break
        props_html = "<b>🔍 Selected Object Properties:</b><br>"
        props = self.visualizer.properties[full_name]
        for key, value in props.items():
            props_html += f"<b>{key}:</b> {value}<br>"
        if element and hasattr(element, 'IsDefinedBy'):
            for rel in element.IsDefinedBy:
                if rel.is_a("IfcRelDefinesByProperties"):
                    pset = rel.RelatingPropertyDefinition
                    if pset.is_a("IfcPropertySet") and hasattr(pset, 'HasProperties'):
                        props_html += f"<br><b>{pset.Name}:</b><br>"
                        for prop in pset.HasProperties:
                            prop_name = getattr(prop, 'Name', 'Unknown')
                            prop_value = getattr(prop, 'NominalValue', None)
                            props_html += f"<b>{prop_name}:</b> {prop_value.wrappedValue if prop_value and hasattr(prop_value, 'wrappedValue') else prop_value if prop_value else 'N/A'}<br>"
        if element and (element.is_a("IfcWall") or element.is_a("IfcWallStandardCase")):
            for rel in self.model.by_type("IfcRelCoversBldgElements"):
                if rel.RelatingBuildingElement == element and rel.RelatedCoverings:
                    for covering in rel.RelatedCoverings:
                        if GeometryExtractor.extract_custom_mesh_from_entity(covering):
                            props_html += f"<br><b>IfcCovering Properties ({covering.GlobalId}):</b><br>"
                            if hasattr(covering, 'IsDefinedBy'):
                                for rel_cov in covering.IsDefinedBy:
                                    if rel_cov.is_a("IfcRelDefinesByProperties"):
                                        pset = rel_cov.RelatingPropertyDefinition
                                        if pset.is_a("IfcElementQuantity") and hasattr(pset, 'Name') and pset.Name.startswith("Qto_"):
                                            for qty in pset.Quantities:
                                                qty_name = getattr(qty, 'Name', 'Unknown')
                                                qty_value = None
                                                if qty.is_a('IfcQuantityLength'):
                                                    qty_value = getattr(qty, 'LengthValue', None)
                                                elif qty.is_a('IfcQuantityArea'):
                                                    qty_value = getattr(qty, 'AreaValue', None)
                                                elif qty.is_a('IfcQuantityVolume'):
                                                    qty_value = getattr(qty, 'VolumeValue', None)
                                                elif qty.is_a('IfcQuantityCount'):
                                                    qty_value = getattr(qty, 'CountValue', None)
                                                elif qty.is_a('IfcQuantityWeight'):
                                                    qty_value = getattr(qty, 'WeightValue', None)
                                                value = qty_value if qty_value is not None else 'N/A'
                                                props_html += f"<b>{qty_name}_Covering:</b> {value}<br>"
                                        elif pset.is_a("IfcPropertySet") and hasattr(pset, 'HasProperties'):
                                            props_html += f"<br><b>{pset.Name} (IfcCovering):</b><br>"
                                            for prop in pset.HasProperties:
                                                prop_name = getattr(prop, 'Name', 'Unknown')
                                                prop_value = getattr(prop, 'NominalValue', None)
                                                props_html += f"<b>{prop_name}:</b> {prop_value.wrappedValue if prop_value and hasattr(prop_value, 'wrappedValue') else prop_value if prop_value else 'N/A'}<br>"
        if not any(k in props for k in props.keys() if k not in ["Name", "Type", "GUID", "Hierarchy"]):
            props_html += "<i>No QTO property available for the main element</i>"
        self.properties_panel.value = props_html

    def _deselect_current(self):
        self.visualizer.selected_mesh[0] = None
        for i, trace in enumerate(self.visualizer.fig.data):
            self.visualizer.fig.data[i].color = self.visualizer.original_colors[trace.name]
        self._update_viewer()
        self.properties_panel.value = "<b>🔍 Properties:</b><br>Select an object"

    def _expand_all_accordions(self, accordion):
        for i in range(len(accordion.children)):
            accordion.selected_index = i
            child = accordion.children[i]
            if hasattr(child, 'children') and isinstance(child.children[0], widgets.Accordion):
                self._expand_all_accordions(child.children[0])

    def _collapse_all_accordions(self, accordion):
        accordion.selected_index = None
        for child in accordion.children:
            if hasattr(child, 'children'):
                for subchild in child.children:
                    if isinstance(subchild, widgets.Accordion):
                        subchild.selected_index = None

 # ==== MAIN FUNCTION ====
def get_ifc_elements_and_visualize(url):
    try:
        model = IFCDownloader.download_and_load(url)
        hierarchy_builder = HierarchicalStructure(model)
        hierarchy = hierarchy_builder.get_hierarchy()
        #hierarchy_builder.print_hierarchy_summary()
        visualizer = Visualizer3D()
        print("\n🔍 Extracting and processing geometry and QTO properties...")
        processed_count = 0
        total_elements = 0
        for storey_name, types in hierarchy.items():
            for ifc_type, elements in types.items():
                total_elements += len(elements)
                for element in elements:
                    mesh_json = GeometryExtractor.extract_custom_mesh_from_entity(element)
                    if mesh_json:
                        qto_props = GeometryExtractor.extract_qto_properties(element, model)
                        hierarchy_path = f"{storey_name}/{ifc_type}"
                        visualizer.add_mesh_from_element(element, mesh_json, hierarchy_path, qto_props)
                        processed_count += 1
                    else:
                        print(f"⚠️ Element {element.GlobalId} ({element.is_a()}) does not have Custom_Mesh")
        #print(f"📊 Total elements in structure: {total_elements}")
        #print(f"📊 Elements processed with geometry: {processed_count}")
        if processed_count == 0:
            print("⚠️ No elements with Custom_Mesh properties found.")
            print("🔍 Check that the IFC file contains Pset_CustomGeometry properties with Custom_Mesh")
            return
        #print(f"✅ {processed_count} elements with geometry processed!")
        visualizer.configure_layout()
        ui = HierarchicalTableUI(hierarchy, visualizer, model)
        ui_widget = ui.create_ui()
        display(ui_widget)
        print("\n✅ Full visualization! Use the table for selection and the panel for visibility.")
    except Exception as e:
        print(f"❌ General error: {e}")
        print(traceback.format_exc())

 # ==== USAGE ====
if __name__ == "__main__":
    url = "https://raw.githubusercontent.com/ionuting/tests/refs/heads/main/output.ifc"
    print("🚀 Starting IFC file processing with hierarchical table, filters, and QTO...")
    print(f"📂 URL: {url}")
    print("-" * 80)
    get_ifc_elements_and_visualize(url)

🚀 Starting IFC file processing with hierarchical table, filters, and QTO...
📂 URL: https://raw.githubusercontent.com/ionuting/tests/refs/heads/main/output.ifc
--------------------------------------------------------------------------------

🔍 Extracting and processing geometry and QTO properties...
⚠️ Element 3wfje55mH3JvI46Wj4j1NM (IfcCovering) does not have Custom_Mesh
⚠️ Element 0ZwZZgHxn4yf6BldHeb9ym (IfcCovering) does not have Custom_Mesh
⚠️ Element 1ARhlb1iH0LeM48qgYmAnE (IfcCovering) does not have Custom_Mesh
⚠️ Element 11KH3IPyHEfwEGIZShYC0V (IfcCovering) does not have Custom_Mesh
⚠️ Element 19Ozt_L8TE3fkIGtAuKItO (IfcCovering) does not have Custom_Mesh
⚠️ Element 1bpYw7S7v6bAxwpm5$c7Vl (IfcCovering) does not have Custom_Mesh
⚠️ Element 3L47X8dVX5herhVMscALmr (IfcCovering) does not have Custom_Mesh
⚠️ Element 0Znm4uehj4kuU5BgBzkqy4 (IfcCovering) does not have Custom_Mesh
⚠️ Element 1u4Zo2jQL9gASAFYaTz1jW (IfcCovering) does not have Custom_Mesh
🔧 Starting UI creation with filter


✅ Full visualization! Use the table for selection and the panel for visibility.
🔍 Applying filters: Storey=All, IFC Type=All
✅ 3D view and table updated
🔍 Applying filters: Storey=EG, IFC Type=All
✅ 3D view and table updated
